In [ ]:
import random

class Room_Effect: #room effects that occur upon entering a room for the first time
    def __init__(self, name, description):
        self.name = name
        self.description = description
    
    def activate_effect(self, character):
        print(f"Upon entering the room, you find: {self.name} - {self.description}")
        if self.name == "the room on fire": #a way to force player to move more
            print(f"{character.name} suffers one light wound if they remain here")
        elif self.name == "the room under lockdown": #prevent player from going back to previous room
            print(f"{character.name} sees the door lock behind them")
        elif self.name == "a specimen staring back at you": #start encounter
            encounter_enemy(character)
        elif self.name == "a place to get your bearings": #neutral room
            print("there is nothing to worry about in this room.")

Room_Effect_Pool = [ #still need to code in effects
    Room_Effect("the room on fire", "if in room at end of turn, suffer one light wound"),
    Room_Effect("the room under lockdown", "unable to move to previous room"),
    Room_Effect("a specimen staring back at you", "starts an encounter"),
    Room_Effect("a place to get your bearings", "nothing happens")
]

class Item: #small and large items to be obtained (hold 2 large, any number of small)
    def __init__(self, name, size = 'small', item_type = None, effect = None):
        self.name = name
        self.size = size 
        self.item_type = item_type
        self.effect = effect
    
    def use(self, character):
        if self.item_type == 'weapon':
            character.attack += self.effect
            print(f"You equip {self.name}. Your attack is now {character.attack}.") 
        elif self.item_type == 'shield':
            character.defense += self.effect
            print(f"You equip {self.name}. Your defense is now {character.defense}.")
        elif self.item_type == 'heal':
            character.heal(self.effect)  

class Character_Inventory: #managable inventory system to add/remove gained items
    def __init__(self):
        self.small_items = [] 
        self.large_items = []
    
    def add_item(self, item):
        if item.size == 'large' and len(self.large_items) < 2:
            self.large_items.append(item)
            print(f"You pick up {item.name}. It looks too heavy for the bag, you'll have to put it in your hand.")
        elif item.size == 'small':
            self.small_items.append(item)
            print(f"You pick up {item.name} and add it to your bag.")
        else:
            print("You can can only carry two large items. You've only got two hands after all.")
    
    def remove_item(self, item):
        if item in self.large_items:
            self.large_items.remove(item)
            print(f"You drop the {item.name}.")
        elif item in self.small_items:
            self.small_items.remove(item)
            print(f"You drop the {item.name}.")
        else:
            print(f"{item.name} is not in your inventory.")
    
    def view_inventory(self): #need to add way for user to add/remove items. Can be done as free action
        print("You peer into your bag, finding: ")
        for item in self.small_items:
            print(f" - {item.name} (Small)")
        for item in self.large_items:
            print(f" - {item.name} (Large)")

small_loot_pool = ["Bandages", "Bullet", "Duct Tape"] #need to figure out more?
large_loot_pool = ["Riot Shield", "Repair Kit", "Pistol"] 

class Room: #create rooms and keep track of whether they've been visited
    def __init__(self, room_id):
        self.room_id = room_id
        self.visited = False
        self.effect = None
        self.small_loot_count = 0
        self.large_loot_count = 0
        self.enemy = None

    def visit(self, character):
        if not self.visited:
            self.visited = True
            self.apply_random_effect(character)
            self.generate_loot()
    
    def apply_random_effect(self, character):
        self.effect = random.choice(Room_Effect_Pool)
        self.effect.activate_effect(character)


    def search_room(self, character):
        if self.small_loot_count > 0 or self.large_loot_count > 0:
            print(f"You search the room...")

            for _ in range(self.small_loot_count):
                loot_item = random.choice(small_loot_pool) 
                character.inventory.add_item(Item(loot_item, 'small'))

            for _ in range(self.large_loot_count):
                loot_item = random.choice(large_loot_pool) 
                character.inventory.add_item(Item(loot_item, 'large'))

            self.small_loot_count = 0
            self.large_loot_count = 0
        else:
            print("You do not find anything else in the room.")

    def generate_loot(self):
        self.small_loot_count = random.randint(1, 3) 
        self.large_loot_count = random.randint(0, 1) 

class Character_Class: #initialize character classes/weapons/stats
    def __init__(self, name, weapon, health, attack, defense):
        self.name = name
        self.weapon = weapon
        self.inventory = Character_Inventory()
        self.health = health
        self.attack = attack
        self.defense = defense
    
    def apply_weapon(self, weapon):
        if weapon:
            self.attack += weapon.effect
            print(f"Weapon equipped: {weapon.name} Attack is now {self.attack}.")
        else:
            print("You decide to continue with what you have. Attack is {self.attack}.")

    def apply_shield(self, shield):
        if shield:
            self.defense += shield.effect
            print(f"You raise your {shield.name}. Defense is now {self.defense}.")
        else:
            print(f"Your arm lays bare. Defense is {self.defense}.")
    
    def takes_damage(self, damage):
        damage_taken = max(0, damage - self.defense)
        self.health -= damage_taken
        if self.health < 0:
            self.health = 0
        print(f"You are hit for {damage_taken} and have {self.health} health remaining.")

    def lives(self):
        return self.health > 0
    
    def heal(self, healing_amount):
        self.health += healing_amount
        print(f"You heal {healing_amount} and now have {self.health} remaining.")
    
    def update_stats(self):
        self.attack = 10
        self.defense = 5
        self.apply_weapon()
        self.apply_shield()

class Character_Creator:
    def create_character(class_name, name):
        class_map = {
            'Engineer' : ('Plasma Cutter', Character_Class, 120, 8, 4),
            'Overseer' : ('Pistol', Character_Class, 100, 12, 5),
            'Scientist' : ('Experimental Rifle', Character_Class, 80, 16, 3),
            'Security Guard' : ('Assault Rifle', Character_Class, 150, 10, 7)
        }

        if class_name in class_map:
            weapon, character_class, health, attack, defense = class_map[class_name]
            return character_class(name, weapon, health, attack, defense)
        else:
            print ("The company does not employ that profession.")
            return None
        
class Enemy:
    def __init__(self, name, health, attack, defense):
        self.name = name
        self.health = health
        self.attack = attack
        self.defense = defense
    
    def takes_damage(self, damage):
        damage_taken = max(0, damage - self.defense)
        self.health -= damage_taken
        if self.health < 0:
            self.health = 0
        print(f"{self.name} is hit for {damage_taken} and has {self.health} health remaining.")
        
    def attacks_player(self, character):
        damage_dealt = max(0, self.attack - character.defense)
        character.health -= damage_dealt
        if character.health < 0:
            character.health = 0
        print(f"{self.name} swings for {damage_dealt} damage. You have {character.health} health left.")
    
    def lives(self):
        return self.health > 0

def encounter_enemy(character):
    global current_room_index
    enemy = Enemy("Specimen", health = random.randint(13, 70), attack = random.randint(10, 20), defense = random.randint(4, 12))
    print(f"A {enemy.name} stares back at you from across the room.")

    while enemy.lives() and character.lives():
        print(f"\n{character.name} - {character.health} health left. \n{enemy.name} - {enemy.health} health left.")
        action_choice = input("\nChoose an action: \n1. Attack \n2. Use Item \n3. Run")

        if action_choice == "1": #attack
            print(f"{character.name} attacks {enemy.name}.")
            enemy.takes_damage(character.attack)
            if enemy.lives():
                enemy.attacks_player(character)
        elif action_choice == "2": #Use Item
            print("Choose an item to use.")
            if character.inventory.small_items:
                print("Your items: ")
                for idx, item in enumerate(character.inventory.small_items, 1):
                    print(f"{idx}. {item.name}")

                item_choice = input("Choose an item to use: ")
                try: 
                    item_idx = int(item_choice) - 1
                    if 0 <= item_idx < len(character.inventory.small_items):
                        item = character.inventory.small_items[item_idx]
                        if item.item_type == "heal":
                            item.use(character)
                            character.inventory.remove_item(item)
                        else:
                            print(f"{item.name} is not usable right now.")
                    else:
                        print("You do not own this item.")
                except ValueError:
                    print("Input not recognized. Try again.")
            else:
                print("Your bag is empty.")
            if enemy.lives():
                enemy.attacks_player(character)
        elif action_choice == "3": #run
            print(f"You attempt to sidestep the {enemy.name} and rush to the next room.")
            escape_chance = random.randint(1, 10)
            success_chance = 7

            if escape_chance >= success_chance:
                print("Your gambit is successful, you successfully make it to the next room.")
                movement = input("Would you like to move forward or backward? (1/2)")
                if movement == '1':
                    if current_room_index < len(rooms) - 1:
                        current_room_index += 1
                        print(f"\nMoving to Room {rooms[current_room_index].room_id}...")
                    else:
                        print("You are at the last room. You can only go backward.")
                elif movement == '2':
                    if current_room_index > 0:
                        current_room_index -= 1
                        print(f"\nMoving to Room {rooms[current_room_index].room_id}...")
                    else:
                        print("You are currently in the first room. You can only go forward.")
                else:
                    print("Please choose between forward and backward.")
            else:
                print(f"The {enemy.name} blocks your exit.")
                enemy.attacks_player(character)
        if not enemy.lives():
            print(f"You stand over the {enemy.name}'s corpse")
            break
        elif not character.lives():
            print(f"Rest in Peace to {character.name}. Here's hoping their successor learns from their mistakes.")
            return

rooms = [Room(i) for i in range(1, 16)] #set of 15 rooms
random.shuffle(rooms) #randomize rooms
current_room_index = 0 #start from first room

def explore_room(character): #room exploration and turn taking
    global current_room_index
    turn_counter = 0 #track actions

    while True: #action loop
        print(f"\nYou are currently in Room {rooms[current_room_index].room_id}.")
        rooms[current_room_index].visit(character)
        
        for _ in range(2):  #action choices, 2 per turn
            print("\nChoose your action:")
            print("1. Search the room")
            print("2. Move to the next room")
            print("3. Move Backward")
            print("4. Stay Put")
            print("5. Check Inventory (NO ACTION USED)")
            print("6. Quit Game")


            action_choice = input("What would you like to do? (1/2/3/4/5/6): ").strip()

            if action_choice == '1': #search room
                rooms[current_room_index].search_room(character)
                turn_counter += 1
                print(f"\nAction {turn_counter} completed: room searched")
            elif action_choice == '2': #move forward
                if current_room_index < len(rooms) - 1:
                    current_room_index += 1
                    print(f"\nMoving to Room {rooms[current_room_index].room_id}...")
                    turn_counter += 1
                else:
                    print("You are at the last room. You can only go backward.")
            elif action_choice == '3': #move backward
                if current_room_index > 0:
                    current_room_index -= 1
                    print(f"\nMoving to Room {rooms[current_room_index].room_id}...")
                    turn_counter += 1
                else:
                    print("You are currently in the first room. You can only go forward.")
            elif action_choice == '4': #end turn
                print("You decide to stay put.")
                break
            elif action_choice == '5': #view inventory
                character.inventory.view_inventory()
                use_item_choice = input("\nWould you like to use anything? (y/n): ").strip().lower()
                if use_item_choice == 'y' or use_item_choice == 'yes':
                    print("Choose an item to use.")
                    if character.inventory.small_items or character.inventory.large_items:
                        print("Your inventory: ")
                        if character.inventory.small_items:
                            print("Small Items:")
                            for idx, item in enumerate(character.inventory.small_items, 1):
                                print(f"{idx}. {item.name}")
                        if character.inventory.large_items:
                            print("Large Items:")
                            for idx, item in enumerate(character.inventory.large_items, 1):
                                print(f"{idx}. {item.name}")
                        
                        item_choice = input("Choose an item number to use (or press Enter to cancel): ").strip()

                        if item_choice: 
                            try:
                                item_idx = int(item_choice) - 1
                                if 0 <= item_idx < len(character.inventory.small_items):
                                    item = character.inventory.small_items[item_idx]
                                    character.use_item(item) 
                                    character.inventory.remove_item(item)  
                                elif 0 <= item_idx < len(character.inventory.large_items):
                                    item = character.inventory.large_items[item_idx]
                                    character.use_item(item)  
                                    character.inventory.remove_item(item) 
                                else:
                                    print("Invalid choice, please select a valid item.")
                            except ValueError:
                                print("Invalid input, please enter a number.")
                    else:
                        print("Your inventory is empty. Nothing to use.")
                else:
                    print("You decide not to use an item.")
            elif action_choice == '6': #break loop and end game
                print("You have abandoned the company.")
                return
            else:
                print("Invalid action. Please choose again.")

        print(f"\nEnd of turn {turn_counter}.")
        input("Press Enter to continue to the next turn...")
        
def class_pick():
    global current_room_index
    while True: #run loop until class is chosen or user chooses to break after selecting wrong
        print("Choose your character: ")
        print("1. Engineer")
        print("2. Overseer") 
        print("3. Scientist") 
        print("4. Security Guard")  

        user_choice = input("Please pick the number corresponding to your desired class: ")

        class_choices = {
            '1': 'Engineer',
            '2' : 'Overseer',
            '3' : 'Scientist',
            '4' : 'Security Guard' 
        }

        if user_choice in class_choices:
            class_name = class_choices[user_choice]
            name = input(f"What's your name?: ")
            character = Character_Creator.create_character(class_name, name)
            final_choice = input(f"So you are {character.name} the {class_name}?").lower()

            if final_choice == 'y' or final_choice == 'yes':
                print(f"Welcome {character.name} the {class_name}! \nPlease take the provided {character.weapon}.") 
                print("The company has been authorized to offer additional support should you seek it, but they require additional tasks to be completed in return.")
                print("The tasks are as follows: ")
                
                explore_room(character) #start exploration post character creation. need to add action system and other actions
                break
        else: 
            print("\nI'm sorry, you don't seem to be in our database.")
            try_again = input("Would you like to try again?: ").lower()

            if try_again != 'y' and try_again != 'yes':
                print ("Server Locking")
                break
class_pick()